## Dynamics analysis

In [ ]:
import json
from pathlib import Path

paths = Path("../env_dynamics").glob("**/*.json")
task_dynamics = {} # webiste -> list of dynamics
for path in paths:
    if "all_env_dynamics" in path.name:
        continue
    website = path.parent.name
    data = json.load(open(path))
    if website not in task_dynamics:
        task_dynamics[website] = [data]
    else:
        task_dynamics[website].extend(data)

In [ ]:
for website_name, dynamics in task_dynamics.items():
    print(f"website: {website_name} has {len(dynamics)} dynamics points")

### Flatten dynamics

In [ ]:
flatten_dynamics = {} # website -> list of dynamics
for website_name, dynamics in task_dynamics.items():
    temp_dynamics = []
    for dynamic in dynamics:
        if isinstance(dynamic, list):
            temp_dynamics.extend(dynamic)
        elif isinstance(dynamic, dict):
            temp_dynamics.append(dynamic)
        else:
            raise ValueError(f"Unknown dynamic type: {type(dynamic)}")
    flatten_dynamics[website_name] = temp_dynamics

In [ ]:
for website_name, dynamics in flatten_dynamics.items():
    print(f"{website_name} has {len(dynamics)} dynamics points")

In [ ]:
with open("../env_dynamics/all_env_dynamics.json", "w") as f:
    json.dump(flatten_dynamics, f, indent=2)

## Clean dynamics

In [ ]:
from openai import OpenAI
import os

client = OpenAI(api_key=os.environ["OPENAI_API_KEY"])
system_prompt = open("prompts/clean_dynamics.txt").read()
model = "o3"

def clean_dynamics(dynamics):
    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": system_prompt}, 
            {"role": "user", "content": "Environment Dynamics: " + json.dumps(dynamics)}
        ],
        temperature=1.0,
        max_completion_tokens=30_000
    )
    return response.choices[0].message.content

In [ ]:
from tqdm import tqdm

filtered_dynamics = {}

for website_name, dynamics in tqdm(flatten_dynamics.items()):
    if website_name in filtered_dynamics.keys():
        print(f"Skipping {website_name} because it already exists")
        continue
    try:
        cleaned_dynamics = clean_dynamics(dynamics)
        try:
            temp_cleaned_json = json.loads(cleaned_dynamics)
        except Exception:
            temp_cleaned_json = cleaned_dynamics
        filtered_dynamics[website_name] = temp_cleaned_json
    except KeyboardInterrupt:
        print("\nKeyboardInterrupt detected. Exiting gracefully.")
        break

In [ ]:
for website_name, dynamics in filtered_dynamics.items():
    print(f"{website_name} has {len(dynamics)} dynamics points")